In [ ]:
import numpy as np
import pandas as pd
import requests
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta

In [113]:
def fetch_binance_data(symbol, interval, limit):
    url = "https://api.binance.com/api/v3/klines"
    params = {'symbol': symbol, 'interval': interval, 'limit': limit}
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data, columns=['open_time', 'open', 'high', 'low', 'close', 'volume',
                                     'close_time', 'quote_asset_volume', 'number_of_trades',
                                     'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['close'] = df['close'].astype(float)
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['volume'] = df['volume'].astype(float)
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    return df

In [114]:
def create_features(df):
    df['SMA_50'] = df['close'].rolling(window=50).mean()
    df['SMA_200'] = df['close'].rolling(window=200).mean()
    df['RSI'] = compute_rsi(df['close'], 14)
    df.dropna(inplace=True)  # Remove rows with NaN values

# Function to calculate Relative Strength Index (RSI)
def compute_rsi(series, period=14):
    delta = series.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [115]:
def train_model(df):
    df['future_close'] = df['close'].shift(-7)  # Predict price 7 days ahead
    df.dropna(inplace=True)  # Remove rows with NaN values

    # Check if there are enough data points for training
    if len(df) < 7:
        print("Not enough data points for training.")
        return None

    features = df[['SMA_50', 'SMA_200', 'RSI']]
    labels = df['future_close']

    X_train = features[:-7]  # Training data
    y_train = labels[:-7]
    X_test = features[-7:]   # Last 7 days for prediction
    y_test = labels[-7:]

    if len(X_train) == 0 or len(y_train) == 0:
        print("Training data is empty. Cannot proceed with model training.")
        return None

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Model Mean Squared Error: {mse:.2f}")

    return model

In [119]:
def predict_future_prices(model, df):
    # Calculate the start date for the next 168 hours
    today = df['close_time'].max()
    next_week_start = today + timedelta(hours=1)  # Start from the next hour

    # Generate future dates for the next 168 hours
    future_dates = pd.date_range(start=next_week_start, periods=168, freq='h')  # Corrected frequency

    # Create future dataframe with the same features
    future_df = pd.DataFrame(index=future_dates)

    # Use last known features to project future features
    last_features = df[['SMA_50', 'SMA_200', 'RSI']].iloc[-1]
    
    # Create an array of future feature values with a trend adjustment
    future_features = np.tile(last_features, (len(future_dates), 1))
    
    # Adjusting the trend projection for simplicity
    future_features[:, 0] += np.arange(len(future_dates)) * 0.01  # Example adjustment for SMA_50
    future_features[:, 1] += np.arange(len(future_dates)) * 0.005  # Example adjustment for SMA_200
    future_features[:, 2] += np.arange(len(future_dates)) * 0.002  # Example adjustment for RSI
    
    future_df[['SMA_50', 'SMA_200', 'RSI']] = future_features
    future_df['predicted_close'] = model.predict(future_df[['SMA_50', 'SMA_200', 'RSI']])
    
    # Return the entire DataFrame including all predicted prices
    return future_df

In [156]:
def main():
    symbol = input("Enter the cryptocurrency symbol (e.g., BTCUSDT): ")
    df = fetch_binance_data(symbol, '1h', 1000)
    create_features(df)

    model = train_model(df)
    if model is None:
        print("Failed to train the model. Exiting.")
        return

    future_df = predict_future_prices(model, df)

    # Convert the 'predicted_close' column into a Pandas Series
    predicted_prices_series = future_df['predicted_close']

    print("\nPredicted Prices for {} in the next 168 hours:".format(symbol))

    # Set the display option to show all rows
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', 1000)

    # Print the predicted prices in a tabular format
    print(future_df[['predicted_close']].to_string(index=True, header=True, float_format=lambda x: '{:.8f}'.format(x)))

    # Reset the display option
    pd.reset_option('display.max_rows')
    pd.reset_option('display.width')

    # Format the time to Hour:Minute
    formatted_times = future_df.index.strftime('%Y-%m-%d %H:%M')

    # Find the best buy and sell times based on the lowest and highest predicted prices
    best_buy_time = future_df.loc[predicted_prices_series.idxmin()].name.strftime('%Y-%m-%d %H:%M')
    best_sell_time = future_df.loc[predicted_prices_series.idxmax()].name.strftime('%Y-%m-%d %H:%M')

    print("\nBest Time to Buy for {}:".format(symbol))
    print("Time: {}, Price: {:.8f}".format(best_buy_time, predicted_prices_series.min()))

    print("Best Time to Sell for {}: ".format(symbol))
    print("Time: {}, Price: {:.8f}".format(best_sell_time, predicted_prices_series.max()))


In [158]:
if __name__ == "__main__":
    main()

Model Mean Squared Error: 45848.22

Predicted Prices for BTCUSDT in the next 168 hours:
                         predicted_close
2024-07-30 07:59:59.999   67070.53260000
2024-07-30 08:59:59.999   67070.53260000
2024-07-30 09:59:59.999   67070.53260000
2024-07-30 10:59:59.999   67070.53260000
2024-07-30 11:59:59.999   67070.53260000
2024-07-30 12:59:59.999   67070.53260000
2024-07-30 13:59:59.999   67070.53260000
2024-07-30 14:59:59.999   67070.53260000
2024-07-30 15:59:59.999   67070.53260000
2024-07-30 16:59:59.999   67070.53260000
2024-07-30 17:59:59.999   67070.53260000
2024-07-30 18:59:59.999   67070.53260000
2024-07-30 19:59:59.999   67070.53260000
2024-07-30 20:59:59.999   67070.53260000
2024-07-30 21:59:59.999   67070.53260000
2024-07-30 22:59:59.999   67070.53260000
2024-07-30 23:59:59.999   67070.53260000
2024-07-31 00:59:59.999   67070.53260000
2024-07-31 01:59:59.999   67070.53260000
2024-07-31 02:59:59.999   67070.53260000
2024-07-31 03:59:59.999   67070.53260000
2024-07-31

: 